# Imports

In [67]:
import cv2
import mediapipe as mp
import numpy as np
import time
from matplotlib import pyplot as plt
import os
import seaborn as sns

# Configuration

In [68]:
BLUE = (255, 0, 0)
GREEN = (0, 255, 0)
RED = (0, 0, 255)
WHITE = (255, 255, 255)


HAND_CONNECTIONS = mp.solutions.hands.HAND_CONNECTIONS
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7,
)

frame_counter = 0

def process_frame(frame, canvas, results, prev_x, prev_y, trajectory_points):
    h, w, _ = frame.shape
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            x, y = int(index_finger_tip.x * w), int(index_finger_tip.y * h)

            cv2.circle(frame, (x, y), 10, RED, -1)

            trajectory_points.append((x, y))

            if prev_x is not None and prev_y is not None:
                cv2.line(canvas, (prev_x, prev_y), (x, y), GREEN, 5)
            prev_x, prev_y = x, y

    cv2.addWeighted(frame, 1, canvas, 1, 0, frame)

    return prev_x, prev_y, frame

# Saving results

In [69]:
def plot_results(results):
    sns.set(style="whitegrid", context="talk")

    metrics = {
        'Avg FPS': [r['avg_fps'] for r in results],
        'Processing Time (s)': [r['processing_time'] for r
                                in results],
        'Trajectory Points': [r['points'] for r in results],
    }

    frame_skips = [r['frame_skip'] for r in results]

    plt.figure(figsize=(18, 5))
    colors = sns.color_palette("Set2", n_colors=3)

    for i, (title, values) in enumerate(metrics.items(), 1):
        plt.subplot(1, 3, i)
        sns.lineplot(x=frame_skips, y=values, marker='o',
                     color=colors[i - 1])
        plt.title(f'{title} vs Frame Skip', fontsize=14)
        plt.xlabel('Frame Skip')
        plt.ylabel(title)
        plt.xticks(frame_skips)
        plt.grid(True)

    plt.tight_layout()
    os.makedirs("results", exist_ok=True)
    plt.savefig("results/plot.png", dpi=300)
    plt.close()
    print("Saved in results/plots.png")


def save_trajectory(points, frame_skip):
    if points:
        os.makedirs("results", exist_ok=True)

        xs, ys = zip(*points)
        plt.figure(figsize=(10, 10))
        plt.plot(xs, ys, 'g-', marker='o', markersize=9, linewidth=6,
                 markerfacecolor='r')
        plt.scatter(xs, ys, color='red', s=9)
        plt.xlim(0, 1920)
        plt.ylim(1080, 0)
        plt.grid(True)
        plt.xlabel('x')
        plt.ylabel('y')
        plt.title(
            f'Trajectory (frame_skip={frame_skip - 1 if frame_skip > 1 else "no skip"})')

        save_path = f'results/trajectory_{frame_skip}.png'
        plt.savefig(save_path)
        plt.close()


def save_results(results):
    os.makedirs("results", exist_ok=True)
    with open("results/experiment_results.md", "w") as f:
        f.write(
            "| Frame Skip | Trajectory Points | Avg FPS | Processing Time (s) | Frames Fed Per Sec | \n")
        f.write(
            "|------------|-------------------|---------|----------------------|----------------------| \n")
        for r in results:
            f.write(
                f"| {r['frame_skip']} | {r['points']} | {r['avg_fps']} | {r['processing_time']} | {int(r['FPS fed per sec'])} | \n")

    print("Saved in results/experiment_results.md")
    plot_results(results)

# Experiment

In [70]:
def run_experiment(frame_skip=1, video_path="video.mp4"):
    cv2.ocl.setUseOpenCL(True)
    prev_x, prev_y = None, None
    frame_counter = 0
    processed_frames_counter = 0
    processing_time_total = 0
    processing_list = []
    trajectory_points = []
    fps_list = []

    cap = cv2.VideoCapture(video_path)
    width, height = int(
        cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(
        cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    canvas = np.zeros((height, width, 3), dtype=np.uint8)
    print("OpenCL enabled:", cv2.ocl.useOpenCL())

    prev_time = time.time()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_counter += 1
        if frame_counter % frame_skip == 0 or frame_counter == 1:
            start = time.process_time()
            rgb_frame = cv2.cvtColor(frame,
                                     cv2.COLOR_BGR2RGB)
            results = hands.process(rgb_frame)
            end = time.process_time()
            processing_time_total += (end - start)
            if processed_frames_counter % (60 / frame_skip) == 0:
                processing_list.append(processing_time_total)
                processing_time_total = 0
            prev_x, prev_y, frame = process_frame(frame,
                                                  canvas,
                                                  results,
                                                  prev_x,
                                                  prev_y,
                                                  trajectory_points)

        current_time = time.time()
        fps = 1 / (current_time - prev_time)
        prev_time = current_time

        cv2.putText(frame, f'FPS: {int(fps)}', (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1,
                    (0, 255, 0), 2)
        fps_list.append(fps)

        # cv2.imshow("Virtual Drawing", frame)

        if cv2.waitKey(int(500 / 60)) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    save_trajectory(trajectory_points, frame_skip)
    return trajectory_points, fps_list, processing_list

def experiments(video_path="video.mp4"):
    frame_skips = [1, 2, 3, 4, 5, 6]
    results = []

    for frame_skip in frame_skips:
        print(f"Running experiment with frame_skip={frame_skip}")
        trajectory_points, fps_list, processing_list = run_experiment(
            frame_skip=frame_skip,
            video_path=video_path
        )
        avg_fps = np.mean(fps_list)
        processing_time = np.sum(processing_list)
        results.append({
            "frame_skip": frame_skip - 1,
            "points": len(trajectory_points),
            "avg_fps": round(avg_fps, 2),
            "processing_time": round(processing_time, 3),
            "FPS fed per sec": int(60 / frame_skip),
        })
    return results

# Run the experiment

In [71]:
results = experiments(video_path="video.mp4")

Running experiment with frame_skip=1
OpenCL enabled: True
Running experiment with frame_skip=2
OpenCL enabled: True
Running experiment with frame_skip=3
OpenCL enabled: True
Running experiment with frame_skip=4
OpenCL enabled: True
Running experiment with frame_skip=5
OpenCL enabled: True
Running experiment with frame_skip=6
OpenCL enabled: True


In [72]:
save_results(results)

Saved in results/experiment_results.md
Saved in results/plots.png
